In [1]:
import pandas as pd

from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from sqlalchemy.orm import sessionmaker

from models import Team
from db_config import get_session
session = get_session()

In [2]:
teams = session.query(Team).all()
session.close()

In [3]:
team_ids = [team.id for team in teams]
print(team_ids)

[1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


In [4]:
def get_2023_2024_by_team_id(team_id):
    try:
        # Create the game finder instance with season and season type specified
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable="2023-24", season_type_nullable="Regular Season")
        
        # Get the data and convert to DataFrame
        games_df = gamefinder.get_data_frames()[0]
        return games_df
        # Display the DataFrame
        print(games_df.head())
    except Exception as e:
        print(f"An error occurred: {e}")
        # Optionally, print out the raw API response for debugging
        print(gamefinder.get_json())



In [5]:
games_by_teams = [get_2023_2024_by_team_id(team_id) for team_id in team_ids]


In [6]:
games = pd.concat(games_by_teams, axis=0)
print(games.head())

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
0     22023  1610612737               ATL  Atlanta Hawks  0022301188   
1     22023  1610612737               ATL  Atlanta Hawks  0022301178   
2     22023  1610612737               ATL  Atlanta Hawks  0022301159   
3     22023  1610612737               ATL  Atlanta Hawks  0022301147   
4     22023  1610612737               ATL  Atlanta Hawks  0022301130   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2024-04-14    ATL @ IND  L  241  115  ...   0.926     9    23   32   25   
1  2024-04-12    ATL @ MIN  L  240  106  ...   0.680     9    31   40   23   
2  2024-04-10  ATL vs. CHA  L  240  114  ...   0.625     7    31   38   35   
3  2024-04-09  ATL vs. MIA  L  292  111  ...   0.786    17    42   59   28   
4  2024-04-06    ATL @ DEN  L  240  110  ...   0.828    11    27   38   30   

   STL  BLK  TOV  PF  PLUS_MINUS  
0    6    5   15  12       -42.0  
1    4    1   14  25        

In [7]:
from datetime import datetime
from models import Game, TeamStats


for index, row in games.iterrows():
    game_date = datetime.strptime(row['GAME_DATE'], '%Y-%m-%d')

    # Check if the game already exists
    game = session.query(Game).filter_by(id=row['GAME_ID']).first()
    if not game:
        game = Game(id=row['GAME_ID'], date=game_date,
                    home_team_id=row['TEAM_ID'] if 'vs.' in row['MATCHUP'] else None,
                    away_team_id=row['TEAM_ID'] if '@' in row['MATCHUP'] else None)
        session.add(game)

    # Check if TeamStats already exists for this game and team
    team_stats = session.query(TeamStats).filter_by(game_id=row['GAME_ID'], team_id=row['TEAM_ID']).first()
    if not team_stats:
        # Create new TeamStats if none exist
        team_stats = TeamStats(
            game=game,
            team_id=row['TEAM_ID']
        )
        session.add(team_stats)

    # Update existing or new TeamStats with current data
    team_stats.points=row['PTS']
    team_stats.field_goals_made=row['FGM']
    team_stats.field_goals_attempted=row['FGA']
    team_stats.field_goal_percentage=row['FG_PCT']
    team_stats.three_point_field_goals_made=row['FG3M']
    team_stats.three_point_field_goals_attempted=row['FG3A']
    team_stats.three_point_field_goal_percentage=row['FG3_PCT']
    team_stats.free_throws_made=row['FTM']
    team_stats.free_throws_attempted=row['FTA']
    team_stats.free_throw_percentage=row['FT_PCT']
    team_stats.offensive_rebounds=row['OREB']
    team_stats.defensive_rebounds=row['DREB']
    team_stats.total_rebounds=row['REB']
    team_stats.assists=row['AST']
    team_stats.steals=row['STL']
    team_stats.blocks=row['BLK']
    team_stats.turnovers=row['TOV']
    team_stats.personal_fouls=row['PF']
    team_stats.plus_minus=row['PLUS_MINUS']

session.commit()
